# Version 1

In [ ]:
# 불용어 제거, 형태소 분석기로 명사 추출, title/content/comments 가중치 부여여

In [17]:
import json
from collections import Counter
from konlpy.tag import Okt
import requests

# 불용어 리스트 가져오기
stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.json"
stopwords = set(requests.get(stopwords_url).json())

# 형태소 분석기 초기화
okt = Okt()

# 가중치 설정
weights = {
    'title': 2.0,
    'content': 1.5,
    'comments': 1.0
}

# 키워드 추출 함수
def extract_keywords(text, weight):
    nouns = okt.nouns(text)
    return Counter({noun: count * weight for noun, count in Counter(nouns).items()})

# JSON 데이터 불러오기
with open("theqoo_crawling_1102_1108.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 각 게시글에 대해 키워드 추출 및 추가
for post in data:
    total_counter = Counter()
    
    # 제목에서 키워드 추출
    if 'title' in post:
        total_counter.update(extract_keywords(post['title'], weights['title']))
    
    # 본문에서 키워드 추출
    if 'content' in post:
        total_counter.update(extract_keywords(post['content'], weights['content']))
    
    # 댓글에서 키워드 추출
    if 'comments' in post and isinstance(post['comments'], list):
        for comment in post['comments']:
            total_counter.update(extract_keywords(comment, weights['comments']))
    
    # 불용어 제거 및 최소 등장 횟수 필터링
    keywords = [word for word, count in total_counter.items() if word not in stopwords and count >= 5]
    
    # 키워드 리스트를 게시글에 추가
    post['keywords'] = keywords

# 결과를 새로운 JSON 파일로 저장
with open("data_with_keywords_v1.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("키워드 추출 및 저장이 완료.")

키워드 추출 및 저장이 완료.


# Version 2

In [19]:
# konply.tag.Okt로 명사 추출
# KMeans 군집화 사용하여 벡터화된 단어 군집화, 각 군집에서 중요 단어 추출
# TfidfVectorizer를 사용하여 군집 내 중요 키워드 추출

In [18]:
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
import json
from konlpy.tag import Okt

# KoBERT 모델 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")

# 형태소 분석기 초기화
okt = Okt()

# KoBERT 임베딩 생성 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# 명사 추출 함수
def extract_nouns(text):
    return okt.nouns(text)

# 데이터 로드
with open("theqoo_crawling_1102_1108.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 게시글 10개만 테스트
data = data[:10]

# 게시글 데이터를 처리
for post in data:
    title = post.get("title", "")
    content = post.get("content", "")
    comments = post.get("comments", [])
    
    # 텍스트에서 명사 추출
    nouns = []
    if title:
        nouns.extend(extract_nouns(title))
    if content:
        nouns.extend(extract_nouns(content))
    for comment in comments:
        nouns.extend(extract_nouns(comment))
    
    # 명사를 벡터화 (KoBERT)
    noun_embeddings = []
    for noun in nouns:
        try:
            embedding = get_embedding(noun)
            noun_embeddings.append((noun, embedding))
        except Exception:
            continue  # KoBERT에서 처리할 수 없는 단어는 무시

    if not noun_embeddings:
        post["keywords"] = []
        continue

    # KoBERT 임베딩 추출
    words, embeddings = zip(*noun_embeddings)
    embeddings = np.vstack(embeddings)

    # KMeans로 군집화
    n_clusters = min(len(embeddings), 3)  # 클러스터 수는 최대 3으로 제한
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)

    # 군집별 단어 빈도 계산
    cluster_nouns = {i: [] for i in range(n_clusters)}
    for word, cluster in zip(words, cluster_labels):
        cluster_nouns[cluster].append(word)

    # 각 클러스터에서 TF-IDF로 상위 10개 키워드 추출
    final_keywords = []
    for cluster, cluster_words in cluster_nouns.items():
        if cluster_words:  # 군집 내 단어가 존재할 경우에만 처리
            vectorizer = TfidfVectorizer()
            try:
                tfidf_matrix = vectorizer.fit_transform(cluster_words)
                sorted_indices = np.argsort(tfidf_matrix.toarray().sum(axis=0))[::-1]
                top_keywords = [vectorizer.get_feature_names_out()[i] for i in sorted_indices[:10]]
                final_keywords.extend(top_keywords)
            except ValueError:
                continue  # 군집 단어가 모두 불용어일 경우 무시

    # 최종 상위 10개의 키워드 선택
    post["keywords"] = list(set(final_keywords))[:10]

# 결과 저장
with open("data_with_keywords_v2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("키워드 추출 및 저장 완료")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


키워드 추출 및 저장 완료


# Version 3

In [ ]:
# 2 대비 title, content, comments 가중치 추가만 다름
# konply.tag.Okt로 명사 추출
# KMeans 군집화 사용하여 벡터화된 단어 군집화, 각 군집에서 중요 단어 추출
# TfidfVectorizer를 사용하여 군집 내 중요 키워드 추출

In [20]:
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
import json
from konlpy.tag import Okt

# KoBERT 모델 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")

# 형태소 분석기 초기화
okt = Okt()

# KoBERT 임베딩 생성 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# 명사 추출 함수
def extract_nouns(text):
    return okt.nouns(text)

# 입력 데이터 로드
input_file = "theqoo_crawling_1102_1108.json"
output_file = "data_with_keywords_v3.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 게시글 10개만 테스트
data = data[:10]

# 가중치 설정
weights = {
    'title': 2.0,
    'content': 1.5,
    'comments': 1.0
}

# 게시글 데이터를 처리
for post in data:
    title = post.get("title", "")
    content = post.get("content", "")
    comments = post.get("comments", [])
    
    # 텍스트에서 명사 추출 및 가중치 적용
    weighted_nouns = []
    if title:
        for noun in extract_nouns(title):
            weighted_nouns.extend([noun] * int(weights['title']))  # title에 가중치 적용
    if content:
        for noun in extract_nouns(content):
            weighted_nouns.extend([noun] * int(weights['content']))  # content에 가중치 적용
    for comment in comments:
        for noun in extract_nouns(comment):
            weighted_nouns.append(noun)  # comments는 기본 가중치
    
    # 명사를 벡터화 (KoBERT)
    noun_embeddings = []
    for noun in weighted_nouns:
        try:
            embedding = get_embedding(noun)
            noun_embeddings.append((noun, embedding))
        except Exception:
            continue  # KoBERT에서 처리할 수 없는 단어는 무시

    if not noun_embeddings:
        post["keywords"] = []
        continue

    # KoBERT 임베딩 추출
    words, embeddings = zip(*noun_embeddings)
    embeddings = np.vstack(embeddings)

    # KMeans로 군집화
    n_clusters = min(len(embeddings), 3)  # 클러스터 수는 최대 3으로 제한
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)

    # 군집별 단어 빈도 계산
    cluster_nouns = {i: [] for i in range(n_clusters)}
    for word, cluster in zip(words, cluster_labels):
        cluster_nouns[cluster].append(word)

    # 각 군집에서 TF-IDF로 상위 10개 키워드 추출
    final_keywords = []
    for cluster, cluster_words in cluster_nouns.items():
        if cluster_words:  # 군집 내 단어가 존재할 경우에만 처리
            vectorizer = TfidfVectorizer()
            try:
                tfidf_matrix = vectorizer.fit_transform(cluster_words)
                sorted_indices = np.argsort(tfidf_matrix.toarray().sum(axis=0))[::-1]
                top_keywords = [vectorizer.get_feature_names_out()[i] for i in sorted_indices[:10]]
                final_keywords.extend(top_keywords)
            except ValueError:
                continue  # 군집 단어가 모두 불용어일 경우 무시

    # 최종 상위 10개의 키워드 선택
    post["keywords"] = list(set(final_keywords))[:10]

# 결과 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"키워드 추출 및 저장 완료")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


키워드 추출 및 저장 완료


# Version 4

In [21]:
# GPT 개선 적용
# 불용어 제거기능 빠져있어서 추가
# 배치 처리와 메모리 최적화: KoBERT 임베딩을 한 번에 처리하도록 최적화하여 속도와 메모리 사용을 개선합니다.
# 에러 로깅: 예외 처리 시 로깅을 추가하여 문제를 추적할 수 있도록 합니다.
# TF-IDF 개선: 군집화 전 전체 데이터에 대해 한 번만 TF-IDF 벡터화를 수행하여 성능을 개선합니다.
# 가독성 향상: 중복 코드를 함수로 정리하여 코드의 가독성을 높입니다.
# 동적 가중치 및 클러스터 수 조정: 동적으로 가중치를 설정하거나 클러스터 수를 조정하여 더 다양한 데이터에 유연하게 대응합니다.

In [23]:
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
import json
from konlpy.tag import Okt
import logging
import requests
import json


# 불용어 리스트를 URL에서 가져오기
stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.json"

# 불용어 목록 로드
response = requests.get(stopwords_url)
STOPWORDS = set(response.json())

# KoBERT 모델 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")

# 형태소 분석기 초기화
okt = Okt()

# KoBERT 임베딩 생성 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# 명사 추출 함수
def extract_nouns(text):
    return okt.nouns(text)

# 입력 데이터 로드
input_file = "theqoo_crawling_1102_1108.json"
output_file = "data_with_keywords_v4.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 게시글 10개만 테스트
data = data[:10]

# 가중치 설정
weights = {
    'title': 2.0,
    'content': 1.5,
    'comments': 1.0
}

# 게시글 데이터를 처리
for post in data:
    title = post.get("title", "")
    content = post.get("content", "")
    comments = post.get("comments", [])
    
    # 텍스트에서 명사 추출 및 가중치 적용
    weighted_nouns = []
    if title:
        for noun in extract_nouns(title):
            weighted_nouns.extend([noun] * int(weights['title']))  # title에 가중치 적용
    if content:
        for noun in extract_nouns(content):
            weighted_nouns.extend([noun] * int(weights['content']))  # content에 가중치 적용
    for comment in comments:
        for noun in extract_nouns(comment):
            weighted_nouns.append(noun)  # comments는 기본 가중치
    
    # 명사를 벡터화 (KoBERT)
    noun_embeddings = []
    for noun in weighted_nouns:
        try:
            embedding = get_embedding(noun)
            noun_embeddings.append((noun, embedding))
        except Exception:
            continue  # KoBERT에서 처리할 수 없는 단어는 무시

    if not noun_embeddings:
        post["keywords"] = []
        continue

    # KoBERT 임베딩 추출
    words, embeddings = zip(*noun_embeddings)
    embeddings = np.vstack(embeddings)

    # KMeans로 군집화
    n_clusters = min(len(embeddings), 3)  # 클러스터 수는 최대 3으로 제한
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)

    # 군집별 단어 빈도 계산
    cluster_nouns = {i: [] for i in range(n_clusters)}
    for word, cluster in zip(words, cluster_labels):
        cluster_nouns[cluster].append(word)

    # 각 군집에서 TF-IDF로 상위 10개 키워드 추출
    final_keywords = []
    for cluster, cluster_words in cluster_nouns.items():
        if cluster_words:  # 군집 내 단어가 존재할 경우에만 처리
            vectorizer = TfidfVectorizer()
            try:
                # cluster_words를 하나의 문자열로 병합
                combined_text = " ".join(cluster_words)
                tfidf_matrix = vectorizer.fit_transform([combined_text])
                sorted_indices = np.argsort(tfidf_matrix.toarray().sum(axis=0))[::-1]
                top_keywords = [vectorizer.get_feature_names_out()[i] for i in sorted_indices[:10]]
                final_keywords.extend(top_keywords)
            except ValueError:
                continue  # 군집 단어가 모두 불용어일 경우 무시

    # 최종 상위 10개의 키워드 선택
    post["keywords"] = list(set(final_keywords))[:10]

# 결과 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"키워드 추출 및 저장 완료")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


키워드 추출 및 저장 완료
